# SQL Summarization with SQLite + OpenAI

This lab is a summarization demo using SQLite for storage and OpenAI for the LLM call.

**Learning objectives**
- Store text data in SQLite
- Query with SQL
- Summarize with an LLM and store results
- Review results with SQL


In [1]:
# Optional: install deps if running in a fresh environment
# !pip install openai python-dotenv


In [2]:
import os
import sqlite3
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))


In [3]:
conn = sqlite3.connect('support_tickets.db')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS tickets (
        ticket_id INTEGER PRIMARY KEY,
        customer_name TEXT,
        category TEXT,
        transcript TEXT,
        summary TEXT
    )
''')

sample_tickets = [
    (1001, 'Alice Brown', 'Delivery', 'The product was delayed by three days and I missed a deadline.'),
    (1002, 'Rajesh Kumar', 'Payment', 'Payment failed but money was deducted; I need a refund.'),
    (1003, 'Maria Lopez', 'Account', 'I cannot reset my password and the link keeps expiring.'),
    (1004, 'Jun Park', 'Product', 'The device overheats after 10 minutes of use.'),
]

cursor.executemany(
    'INSERT OR IGNORE INTO tickets (ticket_id, customer_name, category, transcript) VALUES (?, ?, ?, ?)',
    sample_tickets
)
conn.commit()


In [4]:
# SQL query: find tickets missing summaries
cursor.execute('SELECT ticket_id, transcript FROM tickets WHERE summary IS NULL')
tickets_to_process = cursor.fetchall()
tickets_to_process


[(1001, 'The product was delayed by three days and I missed a deadline.'),
 (1002, 'Payment failed but money was deducted; I need a refund.'),
 (1003, 'I cannot reset my password and the link keeps expiring.'),
 (1004, 'The device overheats after 10 minutes of use.')]

In [5]:
def summarize(text):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {'role': 'system', 'content': 'Summarize this support ticket in one sentence.'},
            {'role': 'user', 'content': text}
        ]
    )
    return response.choices[0].message.content.strip()


In [6]:
for ticket_id, transcript in tickets_to_process:
    summary = summarize(transcript)
    cursor.execute('UPDATE tickets SET summary = ? WHERE ticket_id = ?', (summary, ticket_id))

conn.commit()


In [7]:
cursor.execute('SELECT ticket_id, customer_name, category, summary FROM tickets')
cursor.fetchall()


[(1001,
  'Alice Brown',
  'Delivery',
  'The customer is frustrated because their product was delayed by three days, causing them to miss a deadline.'),
 (1002,
  'Rajesh Kumar',
  'Payment',
  'A customer is requesting a refund because their payment failed but the money was still deducted from their account.'),
 (1003,
  'Maria Lopez',
  'Account',
  'The user is experiencing issues with resetting their password as the reset link keeps expiring.'),
 (1004,
  'Jun Park',
  'Product',
  'The device experiences overheating issues after only 10 minutes of operation.')]